In [1]:
!pip install transformers
import pandas as pd
import numpy as np
from tqdm.auto import tqdm  # progress bar
import tensorflow as tf
from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Tokenize text & create dataset

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

def create_dataset(df, label_name = 'ArgumentLevel', label_number = 5, batch_size = 16, max_length = 256):

    # Tokenize text
    X_input_ids = np.zeros((len(df), max_length))
    X_attn_masks = np.zeros((len(df), max_length))

    for i, text in tqdm(enumerate(df['Content'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length = max_length,
            truncation = True,
            padding = 'max_length',
            add_special_tokens = True, #add [CLS] [PAD] [SEP] tokens
            return_tensors = 'tf'
        )
        
        X_input_ids[i,:] = tokenized_text.input_ids
        X_attn_masks[i,:] = tokenized_text.attention_mask
    

    # Generate labels - One-hot encoding
    labels = np.zeros((len(df), label_number)) 
    labels[np.arange(len(df)), df[label_name].values] = 1


    # Create dataset object
    dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

    # Map dataset
    dataset = dataset.map(DatasetMapFunction)

    # shuffle data
    dataset = dataset.shuffle(400,reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True) # drop remainder 179/16 = 11...3, drop 3 data

    return dataset

Split training and validation data 

In [5]:
def split_dataset(dataset, p = 0.5, batch_size = 16):
    # p - training data size %
    train_size = int((len(df)//batch_size)*p)
    
    train_dataset = dataset.take(train_size) # take the first 8 batches
    val_dataset = dataset.skip(train_size) # take the last 3 batches

    return train_dataset, val_dataset

Create model

In [6]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
def create_model(label_number = 5, max_length = 256):

    # input layer
    input_ids = tf.keras.layers.Input(shape = (max_length), name = 'input_ids', dtype = 'int32')
    # attention layer
    attention_masks = tf.keras.layers.Input(shape = (max_length), name = 'attention_mask', dtype = 'int32')

    # bert layer
    bert_layer = bert_model.bert(input_ids, attention_mask=attention_masks)[1] 

    # intermediate layer
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name = 'intermediate_layer')(bert_layer)

    # output layer - 5 layers because 5 label classes
    output_layer = tf.keras.layers.Dense(label_number, activation='softmax', name = 'output_layer')(intermediate_layer)

    # Create model object
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output_layer)



    # Add optimizer & Loss function & Accuracy metrics
    optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss_func = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy('accuracry')

    model.compile(optimizer = optim, loss = loss_func, metrics = [acc])

    return model


========= 分界线 ==========
==========================

In [8]:
year = 'Y2'
label_name = 'ArgumentLevel'
label_number = 5

run_num = 5

In [9]:
folderpath = '/content/drive/MyDrive/Imperial/nlp-physicseducation/outputs'
outputpath = '/content/drive/MyDrive/Imperial/BERT-results'
filepath = '/sections/labels_cleaned_{year}.csv'.format(year = year)

Load data

In [10]:
df = pd.read_csv(folderpath+filepath)
# df.head()
df['ArgumentLevel'] = df['ArgumentLevel'].replace({'superficial': 0, 'extended': 1, 'deep': 2, 'expert': 3, 'prediction': 4})
df['ReasoningLevel'] = df['ReasoningLevel'].replace({'bal': 0, 'the': 1, 'exp': 2, 'none': 3})
# df['ReasoningLevel'].value_counts()

/Users/jiayangzhang/Library/CloudStorage/GoogleDrive-jiayang.zhang@icloud.com/My Drive/Imperial/nlp-physicseducation/outputs/sections

In [ ]:
train_ratios = [6,7,8,9] 
for s in train_ratios: # training ratio if p=5, train ratio = 0.5
    for i in range(run_num):
        # about dataset
        dataset = create_dataset(df, label_name = label_name, label_number = label_number, batch_size = 16, max_length=512) # create dataset
        train_dataset, val_dataset = split_dataset(dataset, p = (0.1*s), batch_size = 16) # split training and validation data 


        # about model
        model = create_model(label_number = label_number, max_length = 512)   # Create model


        # about train and validate
        history = model.fit(     # Train & Validate model (Fine-tuning)
            train_dataset,
            validation_data = val_dataset,
            epochs = 4
        )


        history_saveto = outputpath + '/{year}/{label_name}/trainsize0{size}/{time}_{year}_{label_name}_{size}trainsize_4epochs_16batchsize_1e-5lr.npy'.format(
            time = datetime.now().strftime("%H%M%S"), 
            year = year, 
            label_name = label_name, 
            size = s
            )
        np.save(history_saveto, history.history) # save history


        # clear model after training
        tf.keras.backend.clear_session()
        import gc
        gc.collect()
        del model

0it [00:00, ?it/s]

Epoch 1/4
3/3 [==============================] - 44s 1s/step - loss: 2.2308 - accuracry: 0.1875 - val_loss: 1.9664 - val_accuracry: 0.1875
Epoch 2/4
3/3 [==============================] - 1s 345ms/step - loss: 1.8922 - accuracry: 0.1875 - val_loss: 1.8004 - val_accuracry: 0.1562
Epoch 3/4
3/3 [==============================] - 1s 345ms/step - loss: 1.6999 - accuracry: 0.0833 - val_loss: 1.6859 - val_accuracry: 0.0625
Epoch 4/4
3/3 [==============================] - 1s 353ms/step - loss: 1.6065 - accuracry: 0.2500 - val_loss: 1.4838 - val_accuracry: 0.3438


0it [00:00, ?it/s]

Epoch 1/4
3/3 [==============================] - 44s 1s/step - loss: 1.7390 - accuracry: 0.3542 - val_loss: 1.4512 - val_accuracry: 0.4062
Epoch 2/4
3/3 [==============================] - 1s 378ms/step - loss: 1.5404 - accuracry: 0.3750 - val_loss: 1.2940 - val_accuracry: 0.4688
Epoch 3/4
3/3 [==============================] - 1s 374ms/step - loss: 1.4197 - accuracry: 0.3958 - val_loss: 1.4120 - val_accuracry: 0.3750
Epoch 4/4
3/3 [==============================] - 1s 372ms/step - loss: 1.3468 - accuracry: 0.3333 - val_loss: 1.3279 - val_accuracry: 0.5000


0it [00:00, ?it/s]

...

Epoch 1/4
3/3 [==============================] - 44s 1s/step - loss: 1.6092 - accuracry: 0.2500 - val_loss: 1.4511 - val_accuracry: 0.3438
Epoch 2/4
3/3 [==============================] - 1s 344ms/step - loss: 1.4771 - accuracry: 0.2708 - val_loss: 1.4237 - val_accuracry: 0.3438
Epoch 3/4
3/3 [==============================] - 1s 343ms/step - loss: 1.4714 - accuracry: 0.3125 - val_loss: 1.3404 - val_accuracry: 0.5625
Epoch 4/4
3/3 [==============================] - 1s 347ms/step - loss: 1.3760 - accuracry: 0.4375 - val_loss: 1.3730 - val_accuracry: 0.5000


0it [00:00, ?it/s]

Epoch 1/4
3/3 [==============================] - 44s 1s/step - loss: 1.9113 - accuracry: 0.1458 - val_loss: 1.7438 - val_accuracry: 0.1875
Epoch 2/4
3/3 [==============================] - 1s 316ms/step - loss: 1.6555 - accuracry: 0.2500 - val_loss: 1.5762 - val_accuracry: 0.1562
Epoch 3/4
3/3 [==============================] - 1s 373ms/step - loss: 1.5598 - accuracry: 0.2292 - val_loss: 1.5695 - val_accuracry: 0.3438
Epoch 4/4
3/3 [==============================] - 1s 326ms/step - loss: 1.4053 - accuracry: 0.5000 - val_loss: 1.3632 - val_accuracry: 0.5625


0it [00:00, ?it/s]

Epoch 1/4


Save model

In [ ]:
model_saveto = './}_{}_trainsize{}'
model.save('Y1Y2_argument_model')

NameError: ignored